In [1]:
#pip install category_encoders

<a href='https://zomato-restaurants-success-prediction-1-0.streamlit.app/'>Zomato_Restaurants_Success_Prediction_app </a>

In [1]:
import warnings 
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
#from sklearn.metrics import confusion_matrix,classification_report
import category_encoders 
from category_encoders import BinaryEncoder,HashingEncoder
from sklearn.preprocessing import OneHotEncoder

import sklearn
sklearn.set_config(transform_output="pandas")
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector,RFE

from sklearn.model_selection import cross_validate
#from sklearn.metrics import accuracy_score,roc_auc_score,auc,precision_score,f1_score,recall_score#,RocCurveDisplay
from sklearn.metrics import RocCurveDisplay

from sklearn.model_selection import GridSearchCV
import joblib as jb

In [2]:
df = pd.read_csv('Zomato_Restaurants_Success_Prediction‬.csv')

In [3]:
df

,name,address,location,listed_in(city),cuisines,rest_type,listed_in(type),dish_liked,menu_item,approx_cost(for two people),online_order,book_table,votes,rate,avg_rate,num_reviews,succeed,menu_item_count,dish_liked_count
0,Jalsa,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Banashankari,Banashankari,"North Indian, Mughlai, Chinese",Casual Dining,Buffet,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...",[],800.0,Yes,Yes,775,4.1,4.08,12,1,0,7
1,Spice Elephant,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Banashankari,Banashankari,"Chinese, North Indian, Thai",Casual Dining,Buffet,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...",[],800.0,Yes,No,787,4.1,3.57,14,1,0,7
2,San Churro Cafe,"1112, Next to KIMS Medical College, 17th Cross...",Banashankari,Banashankari,"Cafe, Mexican, Italian","Cafe, Casual Dining",Buffet,"Churros, Cannelloni, Minestrone Soup, Hot Choc...",[],800.0,Yes,No,918,3.8,3.15,20,1,0,7
3,Addhuri Udupi Bhojana,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Banashankari,Banashankari,"South Indian, North Indian",Quick Bites,Buffet,Masala Dosa,[],300.0,No,No,88,3.7,3.67,35,0,0,1
4,Grand Village,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Basavanagudi,Banashankari,"North Indian, Rajasthani",Casual Dining,Buffet,"Panipuri, Gol Gappe",[],600.0,No,No,166,3.8,4.00,2,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49469,Best Brews - Four Points by Sheraton Bengaluru...,"Four Points by Sheraton Bengaluru, 43/3, White...",Whitefield,Whitefield,Continental,Bar,Pubs and bars,NaN,[],1500.0,No,No,27,3.6,4.12,16,0,0,0
49470,Vinod Bar And Restaurant,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,NaN,[],600.0,No,No,0,0.0,0.00,0,0,0,0
49471,Plunge - Sheraton Grand Bengaluru Whitefield H...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,NaN,[],2000.0,No,No,0,0.0,0.00,0,0,0,0
49472,Chime - Sheraton Grand Bengaluru Whitefield Ho...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,"ITPL Main Road, Whitefield",Whitefield,Finger Food,Bar,Pubs and bars,"Cocktails, Pizza, Buttermilk",[],2500.0,No,Yes,236,4.3,4.00,6,1,0,3


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49474 entries, 0 to 49473
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   name                         49474 non-null  object 
 1   address                      49474 non-null  object 
 2   location                     49453 non-null  object 
 3   listed_in(city)              49474 non-null  object 
 4   cuisines                     49429 non-null  object 
 5   rest_type                    49249 non-null  object 
 6   listed_in(type)              49474 non-null  object 
 7   dish_liked                   23480 non-null  object 
 8   menu_item                    49474 non-null  object 
 9   approx_cost(for two people)  49130 non-null  float64
 10  online_order                 49474 non-null  object 
 11  book_table                   49474 non-null  object 
 12  votes                        49474 non-null  int64  
 13  rate            

In [5]:
df['approx_cost(for two people)'].value_counts()

approx_cost(for two people)
300.0     7117
400.0     6298
500.0     4762
200.0     4511
600.0     3559
          ... 
60.0         1
70.0         1
469.0        1
5000.0       1
3700.0       1
Name: count, Length: 68, dtype: int64

In [6]:
df.isna().sum()

name                               0
address                            0
location                          21
listed_in(city)                    0
cuisines                          45
rest_type                        225
listed_in(type)                    0
dish_liked                     25994
menu_item                          0
approx_cost(for two people)      344
online_order                       0
book_table                         0
votes                              0
rate                               0
avg_rate                           0
num_reviews                        0
succeed                            0
menu_item_count                    0
dish_liked_count                   0
dtype: int64

In [7]:
for col in df.columns[:15]:
    print(df[col].value_counts(),'\n','_'*50)

name
Cafe Coffee Day                                            93
Onesta                                                     85
Just Bake                                                  72
Five Star Chicken                                          70
Empire Restaurant                                          69
                                                           ..
The Mirchi                                                  1
Mast Punjabi Spice                                          1
Maa Biryani Hyderabadi                                      1
Curries 'N' Phulkas                                         1
Plunge - Sheraton Grand Bengaluru Whitefield Hotel &...     1
Name: count, Length: 8463, dtype: int64 
 __________________________________________________
address
Delivery Only                                                                      120
14th Main, 4th Sector, HSR, Bangalore                                               71
The Ritz-Carlton, 99, Residency Road, 

# Drop useless columns in prediction

* I will drop 'dish_liked' column after extracting unique values and save it in a variable and .pkl file ,'rest_type','cuisines' after feature extraction.

In [8]:
df.drop(['name','address','votes','rate','avg_rate','num_reviews','menu_item','menu_item_count'],axis=1 , inplace=True)

In [9]:
df.columns[:15]

Index(['location', 'listed_in(city)', 'cuisines', 'rest_type',
       'listed_in(type)', 'dish_liked', 'approx_cost(for two people)',
       'online_order', 'book_table', 'succeed', 'dish_liked_count'],
      dtype='object')

In [10]:
df.duplicated().sum()

1108

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df.duplicated().sum()

0

In [13]:
df.reset_index(drop=True,inplace=True)

In [14]:
df

,location,listed_in(city),cuisines,rest_type,listed_in(type),dish_liked,approx_cost(for two people),online_order,book_table,succeed,dish_liked_count
0,Banashankari,Banashankari,"North Indian, Mughlai, Chinese",Casual Dining,Buffet,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...",800.0,Yes,Yes,1,7
1,Banashankari,Banashankari,"Chinese, North Indian, Thai",Casual Dining,Buffet,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...",800.0,Yes,No,1,7
2,Banashankari,Banashankari,"Cafe, Mexican, Italian","Cafe, Casual Dining",Buffet,"Churros, Cannelloni, Minestrone Soup, Hot Choc...",800.0,Yes,No,1,7
3,Banashankari,Banashankari,"South Indian, North Indian",Quick Bites,Buffet,Masala Dosa,300.0,No,No,0,1
4,Basavanagudi,Banashankari,"North Indian, Rajasthani",Casual Dining,Buffet,"Panipuri, Gol Gappe",600.0,No,No,1,2
...,...,...,...,...,...,...,...,...,...,...,...
48361,Whitefield,Whitefield,Continental,Bar,Pubs and bars,NaN,1500.0,No,No,0,0
48362,Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,NaN,600.0,No,No,0,0
48363,Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,NaN,2000.0,No,No,0,0
48364,"ITPL Main Road, Whitefield",Whitefield,Finger Food,Bar,Pubs and bars,"Cocktails, Pizza, Buttermilk",2500.0,No,Yes,1,3


In [15]:
df.isna().sum()

location                          19
listed_in(city)                    0
cuisines                          40
rest_type                        223
listed_in(type)                    0
dish_liked                     24918
approx_cost(for two people)      340
online_order                       0
book_table                         0
succeed                            0
dish_liked_count                   0
dtype: int64

In [16]:
df[(df['cuisines'].isna()) | (df['rest_type'].isna()) | (df['location'].isna())].shape

(244, 11)

In [17]:
df

,location,listed_in(city),cuisines,rest_type,listed_in(type),dish_liked,approx_cost(for two people),online_order,book_table,succeed,dish_liked_count
0,Banashankari,Banashankari,"North Indian, Mughlai, Chinese",Casual Dining,Buffet,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...",800.0,Yes,Yes,1,7
1,Banashankari,Banashankari,"Chinese, North Indian, Thai",Casual Dining,Buffet,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...",800.0,Yes,No,1,7
2,Banashankari,Banashankari,"Cafe, Mexican, Italian","Cafe, Casual Dining",Buffet,"Churros, Cannelloni, Minestrone Soup, Hot Choc...",800.0,Yes,No,1,7
3,Banashankari,Banashankari,"South Indian, North Indian",Quick Bites,Buffet,Masala Dosa,300.0,No,No,0,1
4,Basavanagudi,Banashankari,"North Indian, Rajasthani",Casual Dining,Buffet,"Panipuri, Gol Gappe",600.0,No,No,1,2
...,...,...,...,...,...,...,...,...,...,...,...
48361,Whitefield,Whitefield,Continental,Bar,Pubs and bars,NaN,1500.0,No,No,0,0
48362,Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,NaN,600.0,No,No,0,0
48363,Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,NaN,2000.0,No,No,0,0
48364,"ITPL Main Road, Whitefield",Whitefield,Finger Food,Bar,Pubs and bars,"Cocktails, Pizza, Buttermilk",2500.0,No,Yes,1,3


# Feature Extraction

* **Dish_liked** Unique values to save to model to chose menue from and count it

In [18]:
unique_dish_liked = list(df['dish_liked'].value_counts().index)
unique_dish_liked

['Biryani',
 'Friendly Staff',
 'Waffles',
 'Chicken Biryani',
 'Masala Dosa',
 'Paratha',
 'Coffee',
 'Rooftop Ambience',
 'Pizza',
 'Burgers',
 'Cocktails',
 'Prompt Service',
 'Mutton Biryani',
 'Noodles',
 'Cappuccino, Eclair, Pizza, Coffee, Blueberry Cheesecake, Oreo Cheesecake, Tiramisu',
 'Cocktails, Pizza',
 'Decor',
 'Cocktails, Pizza, French Fries',
 'Pizza, Garlic Bread',
 'Breakfast Buffet, Sushi, Cheesecake, Gulab Jamun, Lunch Buffet, Fish, Noodles',
 'Biryani, Breakfast Buffet, Fish, Salads, Burgers, Caramel Pudding, Steak',
 'Waffles, Sandwiches, Nachos, Burgers, Chicken Keema Pizza, Penne Pasta, Smoothie Bowl',
 'Beer',
 'Pasta',
 'Tandoori Chicken',
 'Burgers, Pasta, Chocolate Mousse, Potato Wedges, Cup Cake, Cheesy Fries, Peri Peri Chicken',
 'Burgers, Coffee, Waffles, Mocktails, Pasta, Brownie Chocolate, Chicken Salami',
 'Paneer Peri Peri, Pancakes, Fajitas, Sweet Crepe, Peri Peri Chicken, Chicken Breast, Nutella Crepe',
 'Fish, Curry Crab, Mocktails, Luchi, Kosha M

In [19]:
def dish_liked():
    dishes = set()
    for l in unique_dish_liked:
        try :
            st = l.split(',')
            for i in range(len(st)):
                dishes.add(st[i].strip())
        except:
            dishes.add(l)
    #print(len(dishes),dishes)
    return (dishes)   

In [20]:
menu_items = dish_liked()
menu_items

{'Cucumber Juice',
 'Kaaram Biryani',
 'Cajun Potato',
 'Arugula Salad',
 'Fries Basket',
 'Butterscotch Ice Cream',
 'Tex Mex Burger',
 'Chicken Malai Roll',
 'Rava Masala Dosa',
 'Diwani Handi',
 'Mint Raita',
 'Vegetable Paratha',
 'Chicken Meal',
 'Pudina Chicken',
 'Blueberry Waffles',
 'Idli Sambar',
 'Chicken Filet Burger',
 'Pineapple Lassi',
 'Chicken Kebab Roll',
 'Chicken Seekh Roll',
 'Taco Joint',
 'Sea Food',
 'Hunan Chicken',
 'Tikka Masala Burrito',
 'Tiramisu Cake',
 'Sikandari Raan',
 'Butterfly Chicken',
 'Cheesy Chicken Fries',
 'Masala Vada',
 'Anjal Tava Fry',
 'Bbq Pork Ribs',
 'Pulled Pork Burger',
 'Date Pudding',
 'Grilled Chicken Burger',
 'Alpham Chicken',
 'Chocolate Halwa',
 'Malwani Food',
 'Beef Burger',
 'Plain Paratha',
 'Basil Pannacotta',
 'Chaat Platter',
 'Pumpkin Ravioli',
 'Dum Aloo Kashmiri',
 'Fruit Pancake',
 'Egg Chilli',
 'Garlic Maggi',
 'Herbed Potato Wedges',
 'Seaweed Salad',
 'Bread Pudding',
 'Legendary Arsalan',
 'Kuluki Sarbat',
 'Ru

dish liked uniques will be the list of inputs in streamlit in menu
menu_item_count will be the count of selected items
dish_liked_count will be the count of selected items which is in dish liked uniques

* **drop dish_liked column**

In [21]:
df.drop('dish_liked',axis=1,inplace=True)

* **drop NaNs**

In [22]:
df.isna().sum()

location                        19
listed_in(city)                  0
cuisines                        40
rest_type                      223
listed_in(type)                  0
approx_cost(for two people)    340
online_order                     0
book_table                       0
succeed                          0
dish_liked_count                 0
dtype: int64

In [23]:
df.dropna(inplace=True)

In [24]:
df.isna().sum()

location                       0
listed_in(city)                0
cuisines                       0
rest_type                      0
listed_in(type)                0
approx_cost(for two people)    0
online_order                   0
book_table                     0
succeed                        0
dish_liked_count               0
dtype: int64

In [25]:
df.duplicated().sum()

79

In [26]:
df.drop_duplicates(inplace=True)

In [27]:
df.reset_index(inplace=True,drop=True)

In [28]:
df

,location,listed_in(city),cuisines,rest_type,listed_in(type),approx_cost(for two people),online_order,book_table,succeed,dish_liked_count
0,Banashankari,Banashankari,"North Indian, Mughlai, Chinese",Casual Dining,Buffet,800.0,Yes,Yes,1,7
1,Banashankari,Banashankari,"Chinese, North Indian, Thai",Casual Dining,Buffet,800.0,Yes,No,1,7
2,Banashankari,Banashankari,"Cafe, Mexican, Italian","Cafe, Casual Dining",Buffet,800.0,Yes,No,1,7
3,Banashankari,Banashankari,"South Indian, North Indian",Quick Bites,Buffet,300.0,No,No,0,1
4,Basavanagudi,Banashankari,"North Indian, Rajasthani",Casual Dining,Buffet,600.0,No,No,1,2
...,...,...,...,...,...,...,...,...,...,...
47724,Whitefield,Whitefield,Continental,Bar,Pubs and bars,1500.0,No,No,0,0
47725,Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,600.0,No,No,0,0
47726,Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,2000.0,No,No,0,0
47727,"ITPL Main Road, Whitefield",Whitefield,Finger Food,Bar,Pubs and bars,2500.0,No,Yes,1,3


* **rest_type**	

In [29]:
ml= df.copy()

In [30]:
# to extract the items in rest_type each in acolumn

In [31]:
unique_rest_type = list(df['rest_type'].value_counts().index)
unique_rest_type

['Quick Bites',
 'Casual Dining',
 'Cafe',
 'Delivery',
 'Dessert Parlor',
 'Takeaway, Delivery',
 'Casual Dining, Bar',
 'Bakery',
 'Beverage Shop',
 'Bar',
 'Food Court',
 'Sweet Shop',
 'Bar, Casual Dining',
 'Lounge',
 'Pub',
 'Fine Dining',
 'Casual Dining, Cafe',
 'Beverage Shop, Quick Bites',
 'Bakery, Quick Bites',
 'Mess',
 'Pub, Casual Dining',
 'Sweet Shop, Quick Bites',
 'Kiosk',
 'Cafe, Casual Dining',
 'Cafe, Bakery',
 'Bakery, Dessert Parlor',
 'Cafe, Dessert Parlor',
 'Dessert Parlor, Cafe',
 'Casual Dining, Pub',
 'Dessert Parlor, Quick Bites',
 'Microbrewery, Casual Dining',
 'Cafe, Quick Bites',
 'Takeaway',
 'Quick Bites, Beverage Shop',
 'Food Court, Quick Bites',
 'Food Truck',
 'Dessert Parlor, Beverage Shop',
 'Pub, Microbrewery',
 'Beverage Shop, Dessert Parlor',
 'Bakery, Cafe',
 'Dessert Parlor, Bakery',
 'Quick Bites, Dessert Parlor',
 'Quick Bites, Sweet Shop',
 'Quick Bites, Bakery',
 'Casual Dining, Microbrewery',
 'Confectionery',
 'Microbrewery, Pub',
 

In [32]:
def type():
    types = set()
    col_val=dict()
    for l in unique_rest_type:
        try :
            st = l.split(',')
            for i in range(len(st)):
                
                types.add(st[i].strip().replace(' ','_')+'_type')
                col_val[st[i].strip().replace(' ','_')+'_type'] = st[i].strip()
        except:
            types.add(l)
    #print(types,len(types))
    return col_val , types

In [33]:
types = type()
types[0]

{'Quick_Bites_type': 'Quick Bites',
 'Casual_Dining_type': 'Casual Dining',
 'Cafe_type': 'Cafe',
 'Delivery_type': 'Delivery',
 'Dessert_Parlor_type': 'Dessert Parlor',
 'Takeaway_type': 'Takeaway',
 'Bar_type': 'Bar',
 'Bakery_type': 'Bakery',
 'Beverage_Shop_type': 'Beverage Shop',
 'Food_Court_type': 'Food Court',
 'Sweet_Shop_type': 'Sweet Shop',
 'Lounge_type': 'Lounge',
 'Pub_type': 'Pub',
 'Fine_Dining_type': 'Fine Dining',
 'Mess_type': 'Mess',
 'Kiosk_type': 'Kiosk',
 'Microbrewery_type': 'Microbrewery',
 'Food_Truck_type': 'Food Truck',
 'Confectionery_type': 'Confectionery',
 'Club_type': 'Club',
 'Dhaba_type': 'Dhaba',
 'Irani_Cafee_type': 'Irani Cafee',
 'Meat_Shop_type': 'Meat Shop',
 'Bhojanalya_type': 'Bhojanalya',
 'Pop_Up_type': 'Pop Up'}

In [34]:
def type_cols(df):
    for col,val in types[0].items():
        df[col] = df['rest_type'].apply(lambda x: 1 if val in x else 0)
    return df
    '''for i in list(df.index):
            df.iloc[i,typ] = 1 if typ in df.iloc[i,5] else 0'''

In [35]:
df = type_cols(df)

In [36]:
df

,location,listed_in(city),cuisines,rest_type,listed_in(type),approx_cost(for two people),online_order,book_table,succeed,dish_liked_count,...,Kiosk_type,Microbrewery_type,Food_Truck_type,Confectionery_type,Club_type,Dhaba_type,Irani_Cafee_type,Meat_Shop_type,Bhojanalya_type,Pop_Up_type
0,Banashankari,Banashankari,"North Indian, Mughlai, Chinese",Casual Dining,Buffet,800.0,Yes,Yes,1,7,...,0,0,0,0,0,0,0,0,0,0
1,Banashankari,Banashankari,"Chinese, North Indian, Thai",Casual Dining,Buffet,800.0,Yes,No,1,7,...,0,0,0,0,0,0,0,0,0,0
2,Banashankari,Banashankari,"Cafe, Mexican, Italian","Cafe, Casual Dining",Buffet,800.0,Yes,No,1,7,...,0,0,0,0,0,0,0,0,0,0
3,Banashankari,Banashankari,"South Indian, North Indian",Quick Bites,Buffet,300.0,No,No,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Basavanagudi,Banashankari,"North Indian, Rajasthani",Casual Dining,Buffet,600.0,No,No,1,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47724,Whitefield,Whitefield,Continental,Bar,Pubs and bars,1500.0,No,No,0,0,...,0,0,0,0,0,0,0,0,0,0
47725,Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,600.0,No,No,0,0,...,0,0,0,0,0,0,0,0,0,0
47726,Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,2000.0,No,No,0,0,...,0,0,0,0,0,0,0,0,0,0
47727,"ITPL Main Road, Whitefield",Whitefield,Finger Food,Bar,Pubs and bars,2500.0,No,Yes,1,3,...,0,0,0,0,0,0,0,0,0,0


In [37]:
df['Quick_Bites_type'].value_counts()

Quick_Bites_type
0    29228
1    18501
Name: count, dtype: int64

* **cuisines**

In [38]:
# to extract the items in cuisines each in column

In [39]:
uniqe_cuisines = list(df['cuisines'].value_counts().index)
uniqe_cuisines

['North Indian',
 'North Indian, Chinese',
 'South Indian',
 'Bakery, Desserts',
 'Biryani',
 'Desserts',
 'Cafe',
 'Fast Food',
 'South Indian, North Indian, Chinese',
 'Bakery',
 'Chinese',
 'Ice Cream, Desserts',
 'Chinese, North Indian',
 'Mithai, Street Food',
 'North Indian, Chinese, Biryani',
 'Desserts, Ice Cream',
 'South Indian, North Indian',
 'North Indian, South Indian',
 'North Indian, South Indian, Chinese',
 'Beverages',
 'North Indian, Biryani',
 'Biryani, Kebab',
 'Finger Food',
 'Desserts, Beverages',
 'Street Food',
 'Biryani, North Indian',
 'Beverages, Fast Food',
 'South Indian, Chinese',
 'South Indian, Biryani',
 'Chinese, Momos',
 'Cafe, Fast Food',
 'North Indian, Fast Food',
 'Fast Food, Beverages',
 'Ice Cream',
 'Kerala',
 'Desserts, Bakery',
 'Continental',
 'Bakery, Fast Food',
 'North Indian, Chinese, South Indian',
 'South Indian, North Indian, Chinese, Street Food',
 'North Indian, Mughlai',
 'Burger, Fast Food',
 'Andhra',
 'Fast Food, Rolls',
 'Nort

In [40]:
def cuisine():
    cuisines = set()
    col_val = dict()
    for l in uniqe_cuisines:
        try :
            st = l.split(',')
            for c in st:
                cuisines.add(c.strip().replace(' ','_')+'_cuisine')
                col_val[c.strip().replace(' ','_')+'_cuisine'] = c.strip()
        except:
            cuisines.add(l)
    #print(cuisines,len(cuisines))
    return col_val  ,cuisines

In [41]:
cuisines = cuisine()
cuisines[0]

{'North_Indian_cuisine': 'North Indian',
 'Chinese_cuisine': 'Chinese',
 'South_Indian_cuisine': 'South Indian',
 'Bakery_cuisine': 'Bakery',
 'Desserts_cuisine': 'Desserts',
 'Biryani_cuisine': 'Biryani',
 'Cafe_cuisine': 'Cafe',
 'Fast_Food_cuisine': 'Fast Food',
 'Ice_Cream_cuisine': 'Ice Cream',
 'Mithai_cuisine': 'Mithai',
 'Street_Food_cuisine': 'Street Food',
 'Beverages_cuisine': 'Beverages',
 'Kebab_cuisine': 'Kebab',
 'Finger_Food_cuisine': 'Finger Food',
 'Momos_cuisine': 'Momos',
 'Kerala_cuisine': 'Kerala',
 'Continental_cuisine': 'Continental',
 'Mughlai_cuisine': 'Mughlai',
 'Burger_cuisine': 'Burger',
 'Andhra_cuisine': 'Andhra',
 'Rolls_cuisine': 'Rolls',
 'Thai_cuisine': 'Thai',
 'Pizza_cuisine': 'Pizza',
 'Arabian_cuisine': 'Arabian',
 'Italian_cuisine': 'Italian',
 'Seafood_cuisine': 'Seafood',
 'Bengali_cuisine': 'Bengali',
 'Juices_cuisine': 'Juices',
 'Tibetan_cuisine': 'Tibetan',
 'Healthy_Food_cuisine': 'Healthy Food',
 'Modern_Indian_cuisine': 'Modern Indian',

In [42]:
def cuisines_cols(df):
    for col,val in cuisines[0].items():
        df[col] = df['cuisines'].apply(lambda x: 1 if val in x else 0)
    return df

In [43]:
df = cuisines_cols(df)

In [44]:
df

,location,listed_in(city),cuisines,rest_type,listed_in(type),approx_cost(for two people),online_order,book_table,succeed,dish_liked_count,...,German_cuisine,British_cuisine,Drinks_Only_cuisine,Sindhi_cuisine,Raw_Meats_cuisine,Belgian_cuisine,Tamil_cuisine,Malwani_cuisine,Hot_dogs_cuisine,Pan_Asian_cuisine
0,Banashankari,Banashankari,"North Indian, Mughlai, Chinese",Casual Dining,Buffet,800.0,Yes,Yes,1,7,...,0,0,0,0,0,0,0,0,0,0
1,Banashankari,Banashankari,"Chinese, North Indian, Thai",Casual Dining,Buffet,800.0,Yes,No,1,7,...,0,0,0,0,0,0,0,0,0,0
2,Banashankari,Banashankari,"Cafe, Mexican, Italian","Cafe, Casual Dining",Buffet,800.0,Yes,No,1,7,...,0,0,0,0,0,0,0,0,0,0
3,Banashankari,Banashankari,"South Indian, North Indian",Quick Bites,Buffet,300.0,No,No,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Basavanagudi,Banashankari,"North Indian, Rajasthani",Casual Dining,Buffet,600.0,No,No,1,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47724,Whitefield,Whitefield,Continental,Bar,Pubs and bars,1500.0,No,No,0,0,...,0,0,0,0,0,0,0,0,0,0
47725,Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,600.0,No,No,0,0,...,0,0,0,0,0,0,0,0,0,0
47726,Whitefield,Whitefield,Finger Food,Bar,Pubs and bars,2000.0,No,No,0,0,...,0,0,0,0,0,0,0,0,0,0
47727,"ITPL Main Road, Whitefield",Whitefield,Finger Food,Bar,Pubs and bars,2500.0,No,Yes,1,3,...,0,0,0,0,0,0,0,0,0,0


In [45]:
df['North_Indian_cuisine'].value_counts()

North_Indian_cuisine
0    28475
1    19254
Name: count, dtype: int64

In [46]:
cuisines[0].keys()

dict_keys(['North_Indian_cuisine', 'Chinese_cuisine', 'South_Indian_cuisine', 'Bakery_cuisine', 'Desserts_cuisine', 'Biryani_cuisine', 'Cafe_cuisine', 'Fast_Food_cuisine', 'Ice_Cream_cuisine', 'Mithai_cuisine', 'Street_Food_cuisine', 'Beverages_cuisine', 'Kebab_cuisine', 'Finger_Food_cuisine', 'Momos_cuisine', 'Kerala_cuisine', 'Continental_cuisine', 'Mughlai_cuisine', 'Burger_cuisine', 'Andhra_cuisine', 'Rolls_cuisine', 'Thai_cuisine', 'Pizza_cuisine', 'Arabian_cuisine', 'Italian_cuisine', 'Seafood_cuisine', 'Bengali_cuisine', 'Juices_cuisine', 'Tibetan_cuisine', 'Healthy_Food_cuisine', 'Modern_Indian_cuisine', 'American_cuisine', 'Tea_cuisine', 'Steak_cuisine', 'BBQ_cuisine', 'Japanese_cuisine', 'Mexican_cuisine', 'Maharashtrian_cuisine', 'Asian_cuisine', 'Mangalorean_cuisine', 'European_cuisine', 'Mediterranean_cuisine', 'Vietnamese_cuisine', 'Salad_cuisine', 'Rajasthani_cuisine', 'Sandwich_cuisine', 'Chettinad_cuisine', 'Oriya_cuisine', 'Tex-Mex_cuisine', 'Lucknowi_cuisine', 'Indon

* **drop 'cuisines' , 'rest_type' columns**

In [47]:
df.drop (['cuisines','rest_type'],axis=1,inplace=True) 

In [48]:
df.duplicated().sum()

212

In [49]:
df.drop_duplicates(inplace=True)

In [50]:
df.reset_index(drop=True,inplace=True)

In [51]:
df

,location,listed_in(city),listed_in(type),approx_cost(for two people),online_order,book_table,succeed,dish_liked_count,Quick_Bites_type,Casual_Dining_type,...,German_cuisine,British_cuisine,Drinks_Only_cuisine,Sindhi_cuisine,Raw_Meats_cuisine,Belgian_cuisine,Tamil_cuisine,Malwani_cuisine,Hot_dogs_cuisine,Pan_Asian_cuisine
0,Banashankari,Banashankari,Buffet,800.0,Yes,Yes,1,7,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Banashankari,Banashankari,Buffet,800.0,Yes,No,1,7,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Banashankari,Banashankari,Buffet,800.0,Yes,No,1,7,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Banashankari,Banashankari,Buffet,300.0,No,No,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Basavanagudi,Banashankari,Buffet,600.0,No,No,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47512,Whitefield,Whitefield,Pubs and bars,1500.0,No,No,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47513,Whitefield,Whitefield,Pubs and bars,600.0,No,No,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47514,Whitefield,Whitefield,Pubs and bars,2000.0,No,No,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47515,"ITPL Main Road, Whitefield",Whitefield,Pubs and bars,2500.0,No,Yes,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
df.columns[:20]

Index(['location', 'listed_in(city)', 'listed_in(type)',
       'approx_cost(for two people)', 'online_order', 'book_table', 'succeed',
       'dish_liked_count', 'Quick_Bites_type', 'Casual_Dining_type',
       'Cafe_type', 'Delivery_type', 'Dessert_Parlor_type', 'Takeaway_type',
       'Bar_type', 'Bakery_type', 'Beverage_Shop_type', 'Food_Court_type',
       'Sweet_Shop_type', 'Lounge_type'],
      dtype='object')

In [53]:
df.columns[-20:]

Index(['Grill_cuisine', 'Bubble_Tea_cuisine', 'Portuguese_cuisine',
       'Russian_cuisine', 'Jewish_cuisine', 'Vegan_cuisine',
       'Cantonese_cuisine', 'Bohri_cuisine', 'Afghani_cuisine',
       'Australian_cuisine', 'German_cuisine', 'British_cuisine',
       'Drinks_Only_cuisine', 'Sindhi_cuisine', 'Raw_Meats_cuisine',
       'Belgian_cuisine', 'Tamil_cuisine', 'Malwani_cuisine',
       'Hot_dogs_cuisine', 'Pan_Asian_cuisine'],
      dtype='object')

In [54]:
for col in df.columns:
    print(df[col].value_counts(),'\n','_'*50)

location
BTM                      4399
Koramangala 5th Block    2416
HSR                      2331
JP Nagar                 2035
Indiranagar              1896
                         ... 
Yelahanka                   5
West Bangalore              4
Jakkur                      3
Rajarajeshwari Nagar        2
Peenya                      1
Name: count, Length: 93, dtype: int64 
 __________________________________________________
listed_in(city)
BTM                      2953
Koramangala 7th Block    2710
Koramangala 5th Block    2593
Koramangala 4th Block    2591
Koramangala 6th Block    2446
Jayanagar                2208
JP Nagar                 1889
MG Road                  1723
Church Street            1716
Brigade Road             1699
Indiranagar              1689
Lavelle Road             1661
HSR                      1570
Residency Road           1555
Marathahalli             1481
Whitefield               1445
Bannerghatta Road        1440
Brookefield              1346
Old Airport Ro

Tex-Mex_cuisine
0    47466
1       51
Name: count, dtype: int64 
 __________________________________________________
Lucknowi_cuisine
0    47464
1       53
Name: count, dtype: int64 
 __________________________________________________
Indonesian_cuisine
0    47421
1       96
Name: count, dtype: int64 
 __________________________________________________
Korean_cuisine
0    47377
1      140
Name: count, dtype: int64 
 __________________________________________________
French_cuisine
0    47430
1       87
Name: count, dtype: int64 
 __________________________________________________
Lebanese_cuisine
0    47346
1      171
Name: count, dtype: int64 
 __________________________________________________
Middle_Eastern_cuisine
0    47415
1      102
Name: count, dtype: int64 
 __________________________________________________
Gujarati_cuisine
0    47415
1      102
Name: count, dtype: int64 
 __________________________________________________
Wraps_cuisine
0    47467
1       50
Name: count, dtyp

###### preparing to modeling

In [55]:
df.nunique()[:15]

location                       93
listed_in(city)                30
listed_in(type)                 7
approx_cost(for two people)    68
online_order                    2
book_table                      2
succeed                         2
dish_liked_count                8
Quick_Bites_type                2
Casual_Dining_type              2
Cafe_type                       2
Delivery_type                   2
Dessert_Parlor_type             2
Takeaway_type                   2
Bar_type                        2
dtype: int64

In [56]:
for col in df.columns:
    if df[col].nunique()>2:
        print(f'{col}\t\t {df[col].nunique()}')

location		 93
listed_in(city)		 30
listed_in(type)		 7
approx_cost(for two people)		 68
dish_liked_count		 8


In [57]:
listed_in_types= df['listed_in(type)'].unique()
listed_in_types

array(['Buffet', 'Cafes', 'Delivery', 'Desserts', 'Dine-out',
       'Drinks & nightlife', 'Pubs and bars'], dtype=object)

In [58]:
locations = df['location'].unique()
locations

array(['Banashankari', 'Basavanagudi', 'Mysore Road', 'Jayanagar',
       'Kumaraswamy Layout', 'Rajarajeshwari Nagar', 'Vijay Nagar',
       'Uttarahalli', 'JP Nagar', 'South Bangalore', 'City Market',
       'Nagarbhavi', 'Bannerghatta Road', 'BTM', 'Kanakapura Road',
       'Bommanahalli', 'Electronic City', 'HSR', 'Marathahalli',
       'Wilson Garden', 'Shanti Nagar', 'Koramangala 5th Block',
       'Koramangala 8th Block', 'Richmond Road', 'Koramangala 7th Block',
       'Jalahalli', 'Koramangala 4th Block', 'Bellandur', 'Sarjapur Road',
       'Whitefield', 'East Bangalore', 'Old Airport Road', 'Indiranagar',
       'Koramangala 1st Block', 'Frazer Town', 'RT Nagar', 'MG Road',
       'Brigade Road', 'Lavelle Road', 'Church Street', 'Ulsoor',
       'Residency Road', 'Shivajinagar', 'Infantry Road',
       'St. Marks Road', 'Cunningham Road', 'Race Course Road',
       'Commercial Street', 'Vasanth Nagar', 'HBR Layout', 'Domlur',
       'Ejipura', 'Jeevan Bhima Nagar', 'Old Madr

In [59]:
listed_in_cities = df['listed_in(city)'].unique()
listed_in_cities

array(['Banashankari', 'Bannerghatta Road', 'Basavanagudi', 'Bellandur',
       'Brigade Road', 'Brookefield', 'BTM', 'Church Street',
       'Electronic City', 'Frazer Town', 'HSR', 'Indiranagar',
       'Jayanagar', 'JP Nagar', 'Kalyan Nagar', 'Kammanahalli',
       'Koramangala 4th Block', 'Koramangala 5th Block',
       'Koramangala 6th Block', 'Koramangala 7th Block', 'Lavelle Road',
       'Malleshwaram', 'Marathahalli', 'MG Road', 'New BEL Road',
       'Old Airport Road', 'Rajajinagar', 'Residency Road',
       'Sarjapur Road', 'Whitefield'], dtype=object)

In [60]:
location_each_city = df[['listed_in(city)','location']].value_counts().index
list(location_each_city)

[('Whitefield', 'Whitefield'),
 ('HSR', 'HSR'),
 ('Marathahalli', 'Marathahalli'),
 ('Electronic City', 'Electronic City'),
 ('BTM', 'BTM'),
 ('Indiranagar', 'Indiranagar'),
 ('JP Nagar', 'JP Nagar'),
 ('Koramangala 7th Block', 'BTM'),
 ('Bannerghatta Road', 'Bannerghatta Road'),
 ('Koramangala 5th Block', 'BTM'),
 ('Old Airport Road', 'Indiranagar'),
 ('Jayanagar', 'Jayanagar'),
 ('Bellandur', 'Bellandur'),
 ('Brookefield', 'Whitefield'),
 ('Sarjapur Road', 'Sarjapur Road'),
 ('New BEL Road', 'New BEL Road'),
 ('Jayanagar', 'BTM'),
 ('Koramangala 6th Block', 'BTM'),
 ('Sarjapur Road', 'Bellandur'),
 ('Koramangala 5th Block', 'Koramangala 5th Block'),
 ('Bannerghatta Road', 'JP Nagar'),
 ('Koramangala 4th Block', 'BTM'),
 ('Jayanagar', 'JP Nagar'),
 ('Koramangala 7th Block', 'Koramangala 5th Block'),
 ('Koramangala 6th Block', 'Koramangala 5th Block'),
 ('Banashankari', 'Banashankari'),
 ('Koramangala 4th Block', 'Koramangala 5th Block'),
 ('Brookefield', 'Marathahalli'),
 ('Koramangal

In [61]:
location_dict = dict()

In [62]:
for city in list(listed_in_cities):
    locs=[]
    for ci , loc in list(location_each_city):
        if city.strip() == ci.strip():
            locs.append(loc)
        else:
            pass
    location_dict[city]=locs
        

In [63]:
location_dict

{'Banashankari': ['Banashankari',
  'Basavanagudi',
  'Jayanagar',
  'Kumaraswamy Layout',
  'Mysore Road',
  'Uttarahalli',
  'Vijay Nagar',
  'JP Nagar',
  'South Bangalore',
  'Rajarajeshwari Nagar',
  'Nagarbhavi',
  'City Market'],
 'Bannerghatta Road': ['Bannerghatta Road',
  'JP Nagar',
  'BTM',
  'Bommanahalli',
  'South Bangalore',
  'Jayanagar',
  'Kanakapura Road',
  'Kumaraswamy Layout',
  'Electronic City',
  'Marathahalli',
  'HSR'],
 'Basavanagudi': ['Jayanagar',
  'Basavanagudi',
  'Banashankari',
  'JP Nagar',
  'Shanti Nagar',
  'BTM',
  'City Market',
  'Wilson Garden',
  'Kumaraswamy Layout',
  'Bannerghatta Road',
  'Koramangala 5th Block',
  'Richmond Road',
  'South Bangalore',
  'Koramangala 7th Block',
  'Koramangala 8th Block',
  'Koramangala 4th Block',
  'Jalahalli',
  'HSR',
  'Mysore Road',
  'Nagarbhavi'],
 'Bellandur': ['Bellandur',
  'Sarjapur Road',
  'Marathahalli',
  'HSR',
  'Whitefield',
  'South Bangalore',
  'Koramangala 1st Block',
  'Old Airpor

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47517 entries, 0 to 47516
Columns: 139 entries, location to Pan_Asian_cuisine
dtypes: float64(1), int64(133), object(5)
memory usage: 50.4+ MB


In [65]:
df.describe()

,approx_cost(for two people),succeed,dish_liked_count,Quick_Bites_type,Casual_Dining_type,Cafe_type,Delivery_type,Dessert_Parlor_type,Takeaway_type,Bar_type,...,German_cuisine,British_cuisine,Drinks_Only_cuisine,Sindhi_cuisine,Raw_Meats_cuisine,Belgian_cuisine,Tamil_cuisine,Malwani_cuisine,Hot_dogs_cuisine,Pan_Asian_cuisine
count,47517.000000,47517.000000,47517.000000,47517.000000,47517.000000,47517.000000,47517.000000,47517.000000,47517.00000,47517.000000,...,47517.000000,47517.000000,47517.000000,47517.000000,47517.000000,47517.000000,47517.000000,47517.000000,47517.000000,47517.000000
mean,568.028811,0.444367,2.657028,0.386030,0.263043,0.102237,0.089336,0.063788,0.04047,0.048972,...,0.000147,0.000147,0.000210,0.000084,0.000084,0.000084,0.000126,0.000042,0.000063,0.000021
std,447.439572,0.496901,3.103870,0.486843,0.440290,0.302963,0.285232,0.244377,0.19706,0.215812,...,0.012137,0.012137,0.014506,0.009175,0.009175,0.009175,0.011236,0.006488,0.007946,0.004587
min,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,300.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,400.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,700.000000,1.000000,7.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,6000.000000,1.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [66]:
df.describe(include='O')

,location,listed_in(city),listed_in(type),online_order,book_table
count,47517,47517,47517,47517,47517
unique,93,30,7,2,2
top,BTM,BTM,Delivery,Yes,No
freq,4399,2953,23794,28522,41241


In [67]:
onh = ['online_order','book_table','listed_in(type)']

In [68]:
binary = ['location','listed_in(city)']

### spliting data to X,y

In [69]:
X= df.drop('succeed',axis=1)

In [70]:
X

,location,listed_in(city),listed_in(type),approx_cost(for two people),online_order,book_table,dish_liked_count,Quick_Bites_type,Casual_Dining_type,Cafe_type,...,German_cuisine,British_cuisine,Drinks_Only_cuisine,Sindhi_cuisine,Raw_Meats_cuisine,Belgian_cuisine,Tamil_cuisine,Malwani_cuisine,Hot_dogs_cuisine,Pan_Asian_cuisine
0,Banashankari,Banashankari,Buffet,800.0,Yes,Yes,7,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Banashankari,Banashankari,Buffet,800.0,Yes,No,7,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Banashankari,Banashankari,Buffet,800.0,Yes,No,7,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Banashankari,Banashankari,Buffet,300.0,No,No,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Basavanagudi,Banashankari,Buffet,600.0,No,No,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47512,Whitefield,Whitefield,Pubs and bars,1500.0,No,No,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47513,Whitefield,Whitefield,Pubs and bars,600.0,No,No,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47514,Whitefield,Whitefield,Pubs and bars,2000.0,No,No,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47515,"ITPL Main Road, Whitefield",Whitefield,Pubs and bars,2500.0,No,Yes,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
y=df[['succeed']]

In [72]:
y

,succeed
0,1
1,1
2,1
3,0
4,1
...,...
47512,0
47513,0
47514,0
47515,1


In [73]:
y.value_counts(normalize=True)

succeed
0          0.555633
1          0.444367
Name: proportion, dtype: float64

- target is balanced

# Modeling

### Detecting which model will give best scores (specially : precision& accuracy )

In [75]:
scalers = [
    ('std scaler:', StandardScaler()), 
    ('MinMax scaler :', MinMaxScaler()), 
    ('Robust scaler: ', RobustScaler())
]

In [76]:
#models
models = [
    ('KNN', KNeighborsClassifier()),
    ('Naive', GaussianNB()),
    ('DTC', DecisionTreeClassifier()),
    ('RFC', RandomForestClassifier()),
    ('ADA' ,AdaBoostClassifier()),
    ('GB',GradientBoostingClassifier()),
    ('LogisticR',LogisticRegression()),
    ('SVM', SVC()),      

    ('XGB', XGBClassifier())]
    

In [77]:
# metrics and scores names
sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [78]:
result_models=[] # to append data of scores
for scaler_name, scaler in scalers:
    # print scaler name
    print(f'For {scaler_name} :')
    print('='*30)
    
    # Create the ColumnTransformer preprocessing
    preprocessor =ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder().set_output(transform="pandas") ,binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first').set_output(transform="pandas"), onh)
            ],
        remainder='passthrough'
         )
    
    for model_name, model in models:
        # print model name
        print(f"For {model_name}:")
        # Steps Creation
        stepss = list()
        
        stepss.append(('preprocessor', preprocessor))
        stepss.append((scaler_name, scaler))
        stepss.append((model_name, model))
        
        # Create the pipeline
        pipeline = Pipeline(steps=stepss)
        
        
        ## scoring
        # Perform cross-validation
        # Example evaluation metrics
        cv_results = cross_validate(pipeline, X, y, scoring={'roc_auc': 'roc_auc',
                                                             'accuracy':'accuracy',
                                                             'recall':'recall',
                                                             'precision':'precision',
                                                             'f1':'f1'}# interested scors and metrics
                                                             
                                    , cv=5, return_train_score=True,n_jobs=-1)  # X and y are your input data and targets
        
        
        # Access the results
        test_roc_auc = cv_results['test_roc_auc'].mean()
        train_roc_auc = cv_results['train_roc_auc'].mean()
        print("test_roc_auc :", test_roc_auc)
        print("train_roc_auc:", train_roc_auc)
        print('-'*50)
        test_accuracy = cv_results['test_accuracy'].mean()
        train_accuracy = cv_results['train_accuracy'].mean()
        print("test_accuracy :", test_accuracy)
        print("train_accuracy:", train_accuracy)
        print('-'*50)
        test_recall = cv_results['test_recall'].mean()
        train_recall = cv_results['train_recall'].mean()
        print("test_recall :", test_recall)
        print("train_recall:", train_recall)
        print('-'*50)
        test_precision = cv_results['test_precision'].mean()
        train_precision = cv_results['train_precision'].mean()
        print("test_precision :", test_precision)
        print("train_precision:", train_precision)
        print('-'*50)
        test_f1 = cv_results['test_f1'].mean()
        train_f1 = cv_results['train_f1'].mean()
        print("test_f1 :", test_f1)
        print("train_f1:", train_f1)
        result_models.append([test_roc_auc,train_roc_auc,test_accuracy,train_accuracy, test_recall , train_recall ,
        test_precision,train_precision,test_f1,train_f1])
        print('*' * 50)
    print('*' * 100)
    print('_' * 100)

For std scaler: :
For KNN:
test_roc_auc : 0.9230340711672216
train_roc_auc: 0.9768499543482335
--------------------------------------------------
test_accuracy : 0.8586402362394153
train_accuracy: 0.9200023683723124
--------------------------------------------------
test_recall : 0.8070092351408951
train_recall: 0.87690030783803
--------------------------------------------------
test_precision : 0.865675243892525
train_precision: 0.9390250656165717
--------------------------------------------------
test_f1 : 0.8350615471315361
train_f1: 0.9068950220754843
**************************************************
For Naive:
test_roc_auc : 0.7562908189970028
train_roc_auc: 0.7595599280806812
--------------------------------------------------
test_accuracy : 0.6311635816936774
train_accuracy: 0.632142201282274
--------------------------------------------------
test_recall : 0.2041676533270187
train_recall: 0.20395453469097796
--------------------------------------------------
test_precision : 0.

test_roc_auc : 0.8883852200290597
train_roc_auc: 0.8952733233384519
--------------------------------------------------
test_accuracy : 0.8321654627229182
train_accuracy: 0.836705817140689
--------------------------------------------------
test_recall : 0.767748046412503
train_recall: 0.7753611176888467
--------------------------------------------------
test_precision : 0.8406698616707013
train_precision: 0.8444355409352587
--------------------------------------------------
test_f1 : 0.8024745240548649
train_f1: 0.8084209567347663
**************************************************
For LogisticR:
test_roc_auc : 0.8778386751784819
train_roc_auc: 0.8832293695977527
--------------------------------------------------
test_accuracy : 0.8198751608122332
train_accuracy: 0.8244154859195485
--------------------------------------------------
test_recall : 0.7308548425290078
train_recall: 0.7447549135685531
--------------------------------------------------
test_precision : 0.8428587565504145
train

In [79]:
# result_models to data frame to be clear
results = pd.DataFrame(data=result_models,index=['KNN_std','Naive_std','DTC_std','RFC_std','ADA_std' ,'GB_std','LogisticR_std','SVM_std','XGB_std',
    'KNN_minmax','Naive_minmax','DTC_minmax','RFC_minmax','ADA_minmax' ,'GB_minmax','LogisticR_minmax','SVM_minmax','XGB_minmax',
    'KNN_robust','Naive_robust','DTC_robust','RFC_robust','ADA_robust' ,'GB_robust','LogisticR_robust','SVM_robust','XGB_robust'],
            columns=['test_roc_auc','train_roc_auc','test_accuracy','train_accuracy', 'test_recall' , 'train_recall' ,
                     'test_precision','train_precision','test_f1','train_f1'])

In [80]:
results

,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
KNN_std,0.923034,0.976850,0.858640,0.920002,0.807009,0.876900,0.865675,0.939025,0.835062,0.906895
Naive_std,0.756291,0.759560,0.631164,0.632142,0.204168,0.203955,0.852003,0.865303,0.327716,0.330026
DTC_std,0.905263,0.999914,0.904855,0.993502,0.888515,0.985378,0.896463,1.000000,0.892439,0.992635
RFC_std,0.965867,0.999913,0.916578,0.993497,0.867109,0.988728,0.940518,0.996611,0.902232,0.992654
ADA_std,0.880337,0.884522,0.823895,0.827478,0.762681,0.775266,0.827427,0.825826,0.793662,0.799742
GB_std,0.888386,0.895273,0.832123,0.836706,0.767653,0.775361,0.840653,0.844436,0.802416,0.808421
LogisticR_std,0.877904,0.883481,0.820633,0.824689,0.732607,0.745773,0.843126,0.841674,0.783871,0.790822
SVM_std,0.895771,0.917132,0.839089,0.853231,0.761402,0.776877,0.860314,0.878778,0.807789,0.824690
XGB_std,0.926414,0.955339,0.870615,0.898594,0.807530,0.837177,0.890931,0.927569,0.847056,0.880053
KNN_minmax,0.927692,0.976813,0.867332,0.923548,0.807104,0.869133,0.884760,0.954734,0.843537,0.909915


##  Best 2 models with feature selection

In [74]:
#models
models = [
    ('RFC', RandomForestClassifier()),
    ('XGB', XGBClassifier())]
    

In [75]:
# Initialize RFE feature selector
RFE_selector = RFE(RandomForestClassifier(), n_features_to_select=100) 

# Create the ColumnTransformer preprocessing
encoding = ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder() , binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)],
        remainder='passthrough'
         )
result_models=[] # to append data of scores

for model_name, model in models:
    # print model name
    print(f"For {model_name}:")
    # Steps Creation
    stepss = list()

    stepss.append(('encoding', encoding ))
    stepss.append(('scaling', RobustScaler()))
   
    stepss.append(('feature_selector',RFE_selector))
    stepss.append((model_name, model))

    # Create the pipeline
    pipeline = Pipeline(steps=stepss)


    ## scoring
    # Perform cross-validation
    # Example evaluation metrics
    cv_results = cross_validate(pipeline, X, y, scoring={'roc_auc': 'roc_auc',
                                                         'accuracy':'accuracy',
                                                         'recall':'recall',
                                                         'precision':'precision',
                                                         'f1':'f1'}# interested scors and metrics

                                , cv=5, return_train_score=True,n_jobs=-1)  # X and y are your input data and targets

    # Access the results
    test_roc_auc = cv_results['test_roc_auc'].mean()
    train_roc_auc = cv_results['train_roc_auc'].mean()
    print("test_roc_auc :", test_roc_auc)
    print("train_roc_auc:", train_roc_auc)
    print('-'*50)
    test_accuracy = cv_results['test_accuracy'].mean()
    train_accuracy = cv_results['train_accuracy'].mean()
    print("test_accuracy :", test_accuracy)
    print("train_accuracy:", train_accuracy)
    print('-'*50)
    test_recall = cv_results['test_recall'].mean()
    train_recall = cv_results['train_recall'].mean()
    print("test_recall :", test_recall)
    print("train_recall:", train_recall)
    print('-'*50)
    test_precision = cv_results['test_precision'].mean()
    train_precision = cv_results['train_precision'].mean()
    print("test_precision :", test_precision)
    print("train_precision:", train_precision)
    print('-'*50)
    test_f1 = cv_results['test_f1'].mean()
    train_f1 = cv_results['train_f1'].mean()
    print("test_f1 :", test_f1)
    print("train_f1:", train_f1)
    result_models.append([test_roc_auc,train_roc_auc,test_accuracy,train_accuracy, test_recall , train_recall ,
    test_precision,train_precision,test_f1,train_f1])
    print('*' * 50)
print('*' * 100)
print('_' * 100)

For RFC:
test_roc_auc : 0.9679047614607565
train_roc_auc: 0.9999126188176174
--------------------------------------------------
test_accuracy : 0.9194817045193242
train_accuracy: 0.9934654973607945
--------------------------------------------------
test_recall : 0.874070565948378
train_recall: 0.9884679138053516
--------------------------------------------------
test_precision : 0.9405246851427019
train_precision: 0.9968002893093704
--------------------------------------------------
test_f1 : 0.9060036801299012
train_f1: 0.9926164962847629
**************************************************
For XGB:
test_roc_auc : 0.9275851217282982
train_roc_auc: 0.9579900458667229
--------------------------------------------------
test_accuracy : 0.8712251952334032
train_accuracy: 0.9019456397641935
--------------------------------------------------
test_recall : 0.8082405872602415
train_recall: 0.8412384560738811
--------------------------------------------------
test_precision : 0.8918332872115247
t

In [76]:
# result_models to data frame to be clear
scores = pd.DataFrame(data=result_models,index=['RFC_robust','XGB_robust'],
            columns=['test_roc_auc','train_roc_auc','test_accuracy','train_accuracy', 'test_recall' , 'train_recall' ,
                     'test_precision','train_precision','test_f1','train_f1'])

In [97]:
scores

,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
RFC_robust,0.967905,0.999913,0.919482,0.993465,0.874071,0.988468,0.940525,0.996800,0.906004,0.992616
XGB_robust,0.927585,0.957990,0.871225,0.901946,0.808241,0.841238,0.891833,0.931464,0.847763,0.884052


# Selected model RandomForestClassifier with preprocessors and rubust scaler and RFE for feature selection with n_features_to_select=70

In [77]:
# RFE instance for feature selection
RFE_selector = RFE(RandomForestClassifier(), n_features_to_select=70)

In [78]:
# # Initialize RFE feature selector
# #RFE_selector = RFE(RandomForestClassifier(), n_features_to_select=100) 

# # Create the ColumnTransformer preprocessing
# preprocessing_encoding = Pipeline(
#     steps=[ ColumnTransformer(
#         transformers=[
#             ('binary_encoder', BinaryEncoder() , binary),
#             ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)],
#         remainder='passthrough'
#          )])

In [79]:
# Initialize RFE feature selector
#RFE_selector = RFE(RandomForestClassifier(), n_features_to_select=100) 

# Create the ColumnTransformer preprocessing
encoding = ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder() , binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)],
        remainder='passthrough'
         )

In [80]:
# steps and pipeline for HyperParameter Tuning
steps=[]
steps.append(('encoding', encoding))
  
steps.append(('scaler', RobustScaler()))
steps.append(('feature_selector', RFE_selector))
steps.append(('RFC', RandomForestClassifier()))
pipee=Pipeline(steps=steps)

In [81]:
cv_results = cross_validate(pipee, X, y, scoring={'roc_auc': 'roc_auc',
                                                         'accuracy':'accuracy',
                                                         'recall':'recall',
                                                         'precision':'precision',
                                                         'f1':'f1'}# interested scors and metrics

                                , cv=5, return_train_score=True,n_jobs=-1)

In [82]:
test_roc_auc = cv_results['test_roc_auc'].mean()
train_roc_auc = cv_results['train_roc_auc'].mean()
print("test_roc_auc :", test_roc_auc)
print("train_roc_auc:", train_roc_auc)
print('-'*50)
test_accuracy = cv_results['test_accuracy'].mean()
train_accuracy = cv_results['train_accuracy'].mean()
print("test_accuracy :", test_accuracy)
print("train_accuracy:", train_accuracy)
print('-'*50)
test_recall = cv_results['test_recall'].mean()
train_recall = cv_results['train_recall'].mean()
print("test_recall :", test_recall)
print("train_recall:", train_recall)
print('-'*50)
test_precision = cv_results['test_precision'].mean()
train_precision = cv_results['train_precision'].mean()
print("test_precision :", test_precision)
print("train_precision:", train_precision)
print('-'*50)

test_f1 = cv_results['test_f1'].mean()
train_f1 = cv_results['train_f1'].mean()
print("test_f1 :", test_f1)
print("train_f1:", train_f1)

test_roc_auc : 0.967495983784014
train_roc_auc: 0.999901671051167
--------------------------------------------------
test_accuracy : 0.9179243724558361
train_accuracy: 0.9931024685059997
--------------------------------------------------
test_recall : 0.8712763438313994
train_recall: 0.9881482358512905
--------------------------------------------------
test_precision : 0.9396252707890091
train_precision: 0.9962995286109946
--------------------------------------------------
test_f1 : 0.9040827838927216
train_f1: 0.9922069562502986


* scores have decreased when decreasing n_features_to_select 

## HyperParameter Tuning 

In [105]:
# RFE instance for feature selection
RFE_selector = RFE(RandomForestClassifier(), n_features_to_select=100)

In [106]:
# Initialize RFE feature selector
#RFE_selector = RFE(RandomForestClassifier(), n_features_to_select=100) 

# Create the ColumnTransformer preprocessing
encoding = ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder() , binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)],
        remainder='passthrough'
         )

In [107]:
# steps and pipeline for HyperParameter Tuning
steps=[]
steps.append(('encoding', encoding))
  
steps.append(('scaler', RobustScaler()))
steps.append(('feature_selector', RFE_selector))
steps.append(('RFC', RandomForestClassifier()))
pipee=Pipeline(steps=steps)

In [108]:
param_grid = {
'RFC__max_depth': [None],
'RFC__min_samples_leaf': [1],
'RFC__min_samples_split': [2],

'RFC__n_estimators': [400,500,600] #300,700,1000]

}

In [109]:
# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=pipee, param_grid=param_grid, cv=5, scoring='precision'
                                                             , n_jobs=-1, return_train_score=True)

# Fit the pipeline with GridSearch to the data
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'RFC__max_depth': None, 'RFC__min_samples_leaf': 1, 'RFC__min_samples_split': 2, 'RFC__n_estimators': 400}
Best Score: 0.9413618438796171


In [110]:
# Get the mean test score and mean train score for the best estimator
mean_test_score = grid_search.cv_results_['mean_test_score'][grid_search.best_index_]
mean_train_score = grid_search.cv_results_['mean_train_score'][grid_search.best_index_]

print("Mean Test Score:", mean_test_score)
print("Mean Train Score:", mean_train_score)

Mean Test Score: 0.9413618438796171
Mean Train Score: 0.9980131827683383


In [ ]:
0.9413980780685192
Mean Train Score: 0.998035504783943

In [89]:
grid_search.best_estimator_

Pipeline(steps=[('encoding',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('binary_encoder',
                                                  BinaryEncoder(),
                                                  ['location',
                                                   'listed_in(city)']),
                                                 ('onh_encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['online_order', 'book_table',
                                                   'listed_in(type)'])])),
                ('scaler', RobustScaler()),
                ('feature_selector',
                 RFE(estimator=RandomForestClassifier(),
                     n_features_to_select=70)),
                ('RFC', RandomForestClassifier(n_estimators=400))])

# SAving model with its best parameters

In [111]:
Model = grid_search.best_estimator_

In [112]:
Model.fit(X,y)

Pipeline(steps=[('encoding',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('binary_encoder',
                                                  BinaryEncoder(),
                                                  ['location',
                                                   'listed_in(city)']),
                                                 ('onh_encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['online_order', 'book_table',
                                                   'listed_in(type)'])])),
                ('scaler', RobustScaler()),
                ('feature_selector',
                 RFE(estimator=RandomForestClassifier(),
                     n_features_to_select=100)),
                ('RFC', RandomForestClassifier(n_estimators=400))])

# scoring best params

In [115]:
# RFE instance for feature selection
RFE_selector = RFE(RandomForestClassifier(), n_features_to_select=100)

In [116]:
# Initialize RFE feature selector
#RFE_selector = RFE(RandomForestClassifier(), n_features_to_select=100) 

# Create the ColumnTransformer preprocessing
encoding = ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder().set_output(transform="pandas") , binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first').set_output(transform="pandas"), onh)],
        remainder='passthrough'
         )

In [117]:
# steps and pipeline for HyperParameter Tuning
steps=[]
steps.append(('encoding', encoding))
  
steps.append(('scaler', RobustScaler()))
steps.append(('feature_selector', RFE_selector))
steps.append(('RFC', RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=400)))
pipee=Pipeline(steps=steps)

In [118]:
cv_results = cross_validate(pipee, X, y, scoring={'roc_auc': 'roc_auc',
                                                         'accuracy':'accuracy',
                                                         'recall':'recall',
                                                         'precision':'precision',
                                                         'f1':'f1'}# interested scors and metrics

                                , cv=5, return_train_score=True,n_jobs=-1)

In [119]:
test_roc_auc = cv_results['test_roc_auc'].mean()
train_roc_auc = cv_results['train_roc_auc'].mean()
print("test_roc_auc :", test_roc_auc)
print("train_roc_auc:", train_roc_auc)
print('-'*50)
test_accuracy = cv_results['test_accuracy'].mean()
train_accuracy = cv_results['train_accuracy'].mean()
print("test_accuracy :", test_accuracy)
print("train_accuracy:", train_accuracy)
print('-'*50)
test_recall = cv_results['test_recall'].mean()
train_recall = cv_results['train_recall'].mean()
print("test_recall :", test_recall)
print("train_recall:", train_recall)
print('-'*50)
test_precision = cv_results['test_precision'].mean()
train_precision = cv_results['train_precision'].mean()
print("test_precision :", test_precision)
print("train_precision:", train_precision)
print('-'*50)

test_f1 = cv_results['test_f1'].mean()
train_f1 = cv_results['train_f1'].mean()
print("test_f1 :", test_f1)
print("train_f1:", train_f1)

test_roc_auc : 0.9688245981509078
train_roc_auc: 0.9999134790014775
--------------------------------------------------
test_accuracy : 0.9194396324782781
train_accuracy: 0.9934707585803452
--------------------------------------------------
test_recall : 0.8738337674638881
train_recall: 0.987153682216434
--------------------------------------------------
test_precision : 0.9406585448035584
train_precision: 0.9981324599340372
--------------------------------------------------
test_f1 : 0.9059438622600424
train_f1: 0.9926126316982667


* **These are best scores for the Model**

## Saving important objects For Deploying

In [ ]:
#jb.dump(preprocessing_encoding,"preprocessing.pkl")

In [113]:
jb.dump(Model,"Zomato_Model_Final.pkl",compress=7)

['Zomato_Model_Final.pkl']

In [114]:
jb.dump(X.columns,"Zomato_Columns_Final.pkl")

['Zomato_Columns_Final.pkl']

In [100]:
jb.dump(menu_items ,"Menu_items_liked.pkl")
jb.dump(types[1] ,"rest_type.pkl")
jb.dump(cuisines[1] ,"cuisines.pkl")
jb.dump(locations  ,"locations.pkl")
jb.dump(listed_in_cities ,"listed_in_cities.pkl")
jb.dump(listed_in_types ,"listed_in_types.pkl")

['listed_in_types.pkl']

In [101]:
jb.dump(location_dict ,"location_dict.pkl")

['location_dict.pkl']

In [102]:
list(cuisines[0].values())

['North Indian',
 'Chinese',
 'South Indian',
 'Bakery',
 'Desserts',
 'Biryani',
 'Cafe',
 'Fast Food',
 'Ice Cream',
 'Mithai',
 'Street Food',
 'Beverages',
 'Kebab',
 'Finger Food',
 'Momos',
 'Kerala',
 'Continental',
 'Mughlai',
 'Burger',
 'Andhra',
 'Rolls',
 'Thai',
 'Pizza',
 'Arabian',
 'Italian',
 'Seafood',
 'Bengali',
 'Juices',
 'Tibetan',
 'Healthy Food',
 'Modern Indian',
 'American',
 'Tea',
 'Steak',
 'BBQ',
 'Japanese',
 'Mexican',
 'Maharashtrian',
 'Asian',
 'Mangalorean',
 'European',
 'Mediterranean',
 'Vietnamese',
 'Salad',
 'Rajasthani',
 'Sandwich',
 'Chettinad',
 'Oriya',
 'Tex-Mex',
 'Lucknowi',
 'Indonesian',
 'Korean',
 'French',
 'Lebanese',
 'Middle Eastern',
 'Gujarati',
 'Wraps',
 'Bihari',
 'Goan',
 'Sushi',
 'Kashmiri',
 'Burmese',
 'Hyderabadi',
 'Malaysian',
 'Singaporean',
 'Konkan',
 'Nepalese',
 'North Eastern',
 'Iranian',
 'Parsi',
 'Coffee',
 'Roast Chicken',
 'Greek',
 'Assamese',
 'Afghan',
 'Turkish',
 'Paan',
 'Spanish',
 'Mongolian',
 

In [103]:
# jb.dump(final_model,"Zomato_Model_Final.h5")
# jb.dump(X.columns,"Zomato_Columns_Final.h5")
# jb.dump(menu_items ,"Menu_items_liked.h5")

In [104]:
# jb.dump(X.columns,"Zomato_Columns_Final.h5")
# jb.dump(menu_items ,"Menu_items_liked.h5")

<a href='https://zomato-restaurants-success-prediction-1-0.streamlit.app/'>Zomato_Restaurants_Success_Prediction_app </a>